In [ ]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import random
import os
import json
import scipy
import scipy.stats

# Detectron colors
_COLORS = np.array([
    0.000, 0.447, 0.741,
    0.850, 0.325, 0.098,
    0.929, 0.694, 0.125,
    0.494, 0.184, 0.556,
    0.466, 0.674, 0.188
]).astype(np.float32).reshape((-1, 3))

# Random number generator seed
_RNG_SEED = 1

# Fix RNG seeds
random.seed(_RNG_SEED)
np.random.seed(_RNG_SEED)

# Directory where sweep summaries are stored
_DATA_DIR = '../data'

In [ ]:
def load_sweep(sweep_name):
    """Loads a sweep summary."""
    summary_path = os.path.join(_DATA_DIR, '{}.json'.format(sweep_name))
    with open(summary_path, 'r') as f:
        sweep_summary = json.load(f)
    return sweep_summary

In [ ]:
# Load ResNet sweep
sweep = load_sweep('ResNet')

In [ ]:
# Text experiment, point estimates
random.seed(_RNG_SEED)

num_trials = 5000
N_small = 100
N_large = 1000

small_mins = []
large_mins = []

for i in range(num_trials):
    small = random.sample(sweep, N_small)
    large = random.sample(sweep, N_large)
    small_min = min([j['min_test_top1'] for j in small])
    large_min = min([j['min_test_top1'] for j in large])
    small_mins.append(small_min)
    large_mins.append(large_min)

small_mins = np.array(small_mins)
large_mins = np.array(large_mins)

In [ ]:
small_wins = np.sum((small_mins - large_mins) < 0)
large_wins = np.sum((small_mins - large_mins) > 0)

print('Small wins: {} ({}%)'.format(small_wins, small_wins / num_trials * 100))
print('Large wins: {} ({}%)'.format(large_wins, large_wins / num_trials * 100))

In [ ]:
random.seed(_RNG_SEED)

N_small = 100
N_large = 1000

small = random.sample(sweep, N_small)
large = random.sample(sweep, N_large)

err_small = sorted([j['min_test_top1'] for j in small])
err_large = sorted([j['min_test_top1'] for j in large])

In [ ]:
# Null hypothesis: 2 independent samples are drawn from the same continuous distribution
ks, p_val = scipy.stats.ks_2samp(err_small, err_large)
print(ks, p_val)

In [ ]:
r, c = 1, 1
w, h = 4, 3
fig, axes = plt.subplots(nrows=r, ncols=c, figsize=(c * w, r * h))

#################
# Point estimates
#################
ax = axes

ax.hist(small_mins - large_mins, bins=29, color=_COLORS[0], alpha=0.8)
ax.axvline(0, color=_COLORS[3], alpha=0.8, linewidth=2.5, linestyle='--')

ax.set_xlabel('min_error(B) - min_error(M)', fontsize=16)
ax.set_ylabel('number of values', fontsize=16)
ax.grid(alpha=0.4)

ax.set_xlim([-0.5, 1.0])
ax.set_ylim([0, 500])

plt.tight_layout()

In [ ]:
r, c = 1, 2
l_w, l_h = 4, 3
r_w, r_h = 3, 3
fig, axes = plt.subplots(
    nrows=r, ncols=c,
    figsize=(l_w + r_w, r * l_h),
    gridspec_kw = {'width_ratios':[l_w, r_w]}
)

edf_small = np.arange(N_small) / float(N_small)
edf_large = np.arange(N_large) / float(N_large)

#######################
# Distribution
#######################
ax = axes[0]

ax.plot(
    err_small, edf_small, color=_COLORS[1], linewidth=2, alpha=0.8,
    zorder=1, label='B (n = 100)'
)
ax.plot(
    err_large, edf_large, color=_COLORS[0], linewidth=2, alpha=0.8,
    zorder=0, label='M (n = 1000)'
)

#ax.set_xlim([4.5, 13.5])
ax.set_xlim([4.7, 12.3])
ax.set_xticks([5, 6, 7, 8, 9, 10, 11, 12])
ax.set_xlabel('error', fontsize=16)
ax.set_ylabel('cumulative prob.', fontsize=16)
ax.grid(alpha=0.4)
ax.legend(loc='lower right', prop={'size': 14})

#######################
# Distribution, zoom
#######################
ax = axes[1]

plt.plot(err_small, edf_small, color=_COLORS[1], linewidth=2, alpha=0.8, zorder=1)
plt.plot(err_large, edf_large, color=_COLORS[0], linewidth=2, alpha=0.8, zorder=0)

plt.ylim([0.8, 0.92])
plt.xlim([7.6, 8.6])
plt.xlabel('error', fontsize=16)
plt.grid(alpha=0.4)

plt.tight_layout()